In [1]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
w = Window().orderBy(f.lit('A'))

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/19 23:45:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Leitura de CSV
df0 = spark.read.csv('work/contasapagar.csv',sep=';',header='false');

In [4]:
df1 = df0.select(f.substring(df0._c8,1,2).cast("int").alias('CODIGO'),\
                        f.substring(df0._c8,4,2).cast("int").alias('SUBCODIGO1'),\
                        f.substring(df0._c8,7,2).cast("int").alias('SUBCODIGO2'),\
                        f.substring(f.regexp_replace(df0._c8,'-',''),9,30).alias('DESCSUBCODIGO'),\
                        df0._c0.cast("string").alias('CATEGORIA'),\
                        df0._c1.cast("string").alias('DESCRICAO'),\
                        f.substring(df0._c2,1,2).cast("int").alias('DIAVENCTO'),\
                        f.substring(df0._c2,4,2).cast("int").alias('MESVENCTO'),\
                        f.substring(df0._c2,7,4).cast("int").alias('ANOVENCTO'),\
                        # f.substring(df0._c2,12,2).cast("int").alias('HORAVENCTO'),\
                        # f.substring(df0._c2,15,2).cast("int").alias('MINUTOVENCTO'),\
                        f.substring(df0._c3,1,2).cast("int").alias('DIA_PG'),\
                        f.substring(df0._c3,4,2).cast("int").alias('MES_PG'),\
                        f.substring(df0._c3,7,4).cast("int").alias('ANO_PG'),\
                        # f.substring(df0._c3,12,2).cast("int").alias('HORAPG'),\
                        # f.substring(df0._c3,15,2).cast("int").alias('MINUTOPG'),\
                        f.regexp_replace(f.regexp_replace(f.regexp_replace(f.trim(df0._c4),'[R$ ]',''),'[.]',''),',','.').alias('VALOR'),\
                        f.regexp_replace(f.regexp_replace(f.regexp_replace(f.trim(df0._c5),'[R$ ]',''),'[.]',''),',','.').alias('VALORPG'),\
                        df0._c6.cast("string").alias('STATUS'),\
                        df0._c7.cast("string").alias('PARCELAS'),\
                        df0._c8.cast("string").alias('FORNECEDOR'),\
                        df0._c9.cast("string").alias('UNIDADE'))
print(f'Contas a Pagar Formatados : {df1.count()}')

Contas a Pagar Formatados : 547


In [5]:
# df1.filter(df1.DESCRICAO == "FRANQUIA").show()

In [6]:
df2 = df1.distinct()

In [7]:
# print(f'Contas a Pagar Formatados : {df2.count()}')
# df2.show()

In [8]:
df3 = df2.dropna(how='all', subset='DESCRICAO')
print(f'Contas a Pagar : {df3.count()}')

Contas a Pagar : 544


In [9]:
df_contasapagar = df3.select(f.row_number().over(w).alias('ID'),'*')

In [10]:
# df_contasapagar.show(truncate=False)

In [11]:
df_cotas = df_contasapagar.select('CODIGO','SUBCODIGO1','SUBCODIGO2','DESCSUBCODIGO').distinct().dropna(how='all', subset='CODIGO')
df_cotas.write.format("parquet").mode("overwrite").parquet('work/codigocontas')

In [12]:
df_contasapagar.write.format("parquet").mode("overwrite").parquet('work/contasapagar')

22/06/19 23:46:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:46:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:46:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:46:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [13]:
# dfMelhorDia = df_contasapagar.groupBy("MES_PG","CATEGORIA").agg(f.format_number(f.sum("VALORPG"),2).alias("total"),\
#                              f.format_number(f.avg("VALORPG"),2).alias("media"),\
#                              f.format_number(f.min("VALORPG"),2).alias("menor"),\
#                              f.format_number(f.max("VALORPG"),2).alias("maior")).\
#     orderBy(["MES_PG","total"],ascending=[0, 0])

In [23]:
dfMelhorDia = df_contasapagar.groupBy("MES_PG","CODIGO","SUBCODIGO1","CATEGORIA").agg(f.round(f.sum("VALORPG"),2).alias("total"),\
                             f.count("MES_PG").alias("Compras"),\
                             f.round(f.avg("VALORPG"),2).alias("media"),\
                             f.round(f.min("VALORPG"),2).alias("menor"),\
                             f.round(f.max("VALORPG"),2).alias("maior")).\
    orderBy(["MES_PG",'CODIGO','SUBCODIGO1',"total"],ascending=[1, 1,1])

In [25]:
dfMelhorDia = df_contasapagar.groupBy("MES_PG","CODIGO","SUBCODIGO1","CATEGORIA").agg(f.round(f.sum("VALORPG"),2).alias("total"),\
                             f.count("MES_PG").alias("Compras")).\
    orderBy(["MES_PG",'CODIGO','SUBCODIGO1',"total"],ascending=[1, 1,1])

In [26]:
dfMelhorDia.show(truncate=False,n=1000)


+------+------+----------+-----------------------------------+--------+-------+
|MES_PG|CODIGO|SUBCODIGO1|CATEGORIA                          |total   |Compras|
+------+------+----------+-----------------------------------+--------+-------+
|null  |null  |null      |03.03-Instalações Gerais           |null    |0      |
|null  |null  |null      |Catagoria                          |null    |0      |
|null  |1     |0         |01.00-Aluguel                      |null    |0      |
|null  |1     |1         |01.01-SABESP                       |null    |0      |
|null  |1     |2         |01.02-Energia Elétrica             |null    |0      |
|5     |null  |null      |03.01-Itens para Cozinha           |89.26   |2      |
|5     |null  |null      |01.11-Delivery                     |248.0   |5      |
|5     |null  |null      |01.15-Mercado                      |227.0   |6      |
|5     |null  |null      |03.03-Instalações Gerais           |10.0    |1      |
|5     |null  |null      |02.08-Vale Tra

22/06/20 17:45:58 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1577176 ms exceeds timeout 120000 ms
22/06/20 17:45:58 WARN SparkContext: Killing executors is not supported by current scheduler.
